# What is Convolutional LSTM Model


*   Convolutional LSTM is a neural network that combines the Convolutional Neural Network (CNN) and Long Short-Term Memory (LSTM) models to handle spatiotemporal data.
*   CNN used to capture the spatial dependencies in traffic flow
data.
*   LSTM used to capture short-term changes and periodic patterns in
traffic flow. The LSTM structure helps in handling long-term
dependencies. Includes daily and
weekly periodic features using LSTMs to capture long-term
and short-term variations.
*   Feature-Level Fusion is combines features from CNN and LSTM and helps improve performance.


# Necessary Modules

Imports all required libraries for data processing, model building, and evaluation.

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model ,save_model,load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Flatten, concatenate, Bidirectional

# Data Loading and Preprocessing

Loads CSV files from different folders, processes them, and combines them into a single DataFrame, ensuring no missing values.

In [ ]:
folder_paths = ["Station - 1","Station - 2","Station - 3","Station - 4"]
dfs_per_folder = []
for idx, folder_path in enumerate(folder_paths, start=1):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    files = [f for f in files if f != 'combined.csv']
    files.sort(key=lambda x: int(os.path.splitext(x)[0]))
    dfs = []
    for file in files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        df.columns = [f"{col}_{idx}" for col in df.columns]
        dfs.append(df)
    combined_df = pd.concat(dfs, ignore_index=True)
    dfs_per_folder.append(combined_df)
final_combined_df = pd.concat(dfs_per_folder, axis=1)

In [ ]:
final_combined_df = final_combined_df.iloc[:-3]
df = pd.DataFrame(final_combined_df)
final_combined_df = df.dropna()
final_combined_df

# Normalization

Scales the target columns using MinMaxScaler to bring data into a range between 0 and 1.

In [ ]:
our_particular_station = ['Station - 3']
station_scaler = MinMaxScaler()
final_combined_df[column1] = station_scaler.fit_transform(final_combined_df[our_particular_station])

In [ ]:
flow_columns = ['5 Minutes_1', "Station - 1","Station - 2","Station - 3","Station - 4"]
flows_df = final_combined_df[flow_columns].copy()
flows_df.rename(columns={'5 Minutes_1': '5 Minutes'}, inplace=True)

# Sequence creation

Converts the time series data into sequences for CNN and LSTM, allowing models to learn patterns over time steps.

In [ ]:
def create_sequences(column_data, time_steps):
    X, Y = [], []
    for i in range(len(column_data) - time_steps):
        X.append(column_data[i:i + time_steps])
        Y.append(column_data[i + time_steps])
    return np.array(X), np.array(Y)

column_data = flows_df[["Station - 1","Station - 2","Station - 3","Station - 4"]].values
time_steps = 15

cnn_X, _ = create_sequences(column_data, time_steps)

station1_data = column_data[:, 0].reshape(-1, 1)
lstm_X, y = create_sequences(station1_data, time_steps)

print(f"Values of cnn_X: {cnn_X}")
print(f"Values of y: {y}")

print(f"Shape of cnn_X: {cnn_X.shape}")
print(f"Shape of y: {y.shape}")

# Data Spliting

Splits the data into training and testing sets for both CNN and LSTM inputs.

In [ ]:
train_size = int(len(cnn_X) * 0.85)
test_size = len(cnn_X) - train_size

X_train_cnn, X_test_cnn = cnn_X[:train_size], cnn_X[train_size:]
X_train_lstm, X_test_lstm = lstm_X[:train_size], lstm_X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"Training data size: {train_size}, Testing data size: {test_size}")

print("Shapes of the data:")
print(f"X_train_cnn shape: {X_train_cnn.shape}")
print(f"X_test_cnn shape: {X_test_cnn.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

print("Training CNN X Values : ",X_train_cnn)
print("Testing CNN X Values : ",X_test_cnn)
print("Training y Values : ",y_train)
print("Testing y Values : ",y_test)

# CONV-LSTM Model

Defines a Convolutional LSTM model that combines CNN for spatial features and LSTM for temporal features, with a final dense layer for predictions.

In [ ]:
cnn_input = Input(shape=(15, 4))

conv1 = Conv1D(filters=32,kernel_size=8,strides=1,activation='tanh', padding='same')(cnn_input) # Add padding='same'
pool1 = MaxPooling1D(pool_size=4)(conv1)

conv2 = Conv1D(filters=16,kernel_size=4,strides=1,activation='tanh', padding='same')(pool1) # Add padding='same'
pool2 = MaxPooling1D(pool_size=2)(conv2)

lstm1 = LSTM(100, return_sequences=True)(pool2)
lstm2 = LSTM(50)(lstm1)

output_layer = Dense(1)(lstm2)

model = Model(inputs=cnn_input, outputs=output_layer)

model.compile(optimizer='RMSprop', loss='mean_squared_error')

model.summary()

# Training the CONV-LSTM Model

Trains the model on the training data for 100 epochs with a validation split of 20%.

In [ ]:
history = model.fit(X_train_cnn, y_train, epochs=100, validation_split=0.2)

# Predictions and Evaluation

Makes predictions on the test set and rescales them to their original values.

In [ ]:
predicted_values = model.predict(X_test_cnn)
predictions_rescaled = station_scaler.inverse_transform(predicted_values.reshape(-1, 1)).flatten()
y_test_rescaled = station_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
results = pd.DataFrame(data={'Predictions': predictions_rescaled, 'Actuals': y_test_rescaled})
print(results)

# Evaluation Metrics

Calculates RMSE, MAE, and MAPE to evaluate the model's performance.

In [ ]:
np.isnan(predictions_rescaled).any()
np.isnan(y_test_rescaled).any()

predictions_rescaled = np.nan_to_num(predictions_rescaled, nan=np.nanmean(predictions_rescaled))
y_test_rescaled = np.nan_to_num(y_test_rescaled, nan=np.nanmean(y_test_rescaled))

rmse = math.sqrt(mean_squared_error(predictions_rescaled,y_test_rescaled))
mae = mean_absolute_error(y_test_rescaled, predictions_rescaled)
mape = mean_absolute_percentage_error(predictions_rescaled, y_test_rescaled)
print ("RMSE:", rmse)
print ("MAE:", mae)
print ("MAPE:", mape)

# Save and Load the Model

Saves the trained model to a file and demonstrates how to load it for future use.

In [ ]:
model.save('CONV-LSTM-Model.h5')
loaded_model = load_model('CONV-LSTM-Model.h5')

# Loss Graph

Plots the training and validation loss over epochs to visualize model convergence.

In [ ]:
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'Validation'], loc='upper left')
plt.show()